In [120]:
%load_ext autoreload
%autoreload 2

import torch
import tqdm
import gpytorch


import numpy as np
from ase import io
import matplotlib.pyplot as plt

In [1]:
import sys
sys.path.append("../../fande") # Adds higher directory to python modules path.
sys.path.append("..") # Adds higher directory to python modules path.

In [100]:
path = os.getcwd()

print(path)

/home/dlb/coding/shared_coding/repos/chem-gp/saddle-dynamics/notebooks


In [4]:
from fande.data import FandeDataModule

fande.models module imported...
FandE-GP imported  
objects of fande.data module imported...


### Parsing and loading data:

In [15]:
%load_ext autoreload
%autoreload 2

from sdynamics.load import parse_trajectories, parse_forces, flatten_trj_dictionaries

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
trajectories, energies_trj, trj_files_basenames = parse_trajectories(traj_folder="../data/dynamics/ene_grad_fitting/data/trj/")
forces_trj = parse_forces(forces_path = '../data/dynamics/ene_grad_fitting/data/grad/', trj_files_basenames = trj_files_basenames)

Reading trajectory files...


100%|██████████| 50/50 [00:17<00:00,  2.89it/s]

Trajectory files reading done!
Reading .npy files with forces...
Reading .npy files with forces done!


In [33]:
traj, energies, forces = flatten_trj_dictionaries(trajectories, energies_trj, trj_files_basenames, forces_trj)

Flattening done!


### Training simple GP

In [49]:
from fande.ase.gp_model import GP_model_ase_traj

training_data = {'traj': traj, 'energies': energies, 'forces': forces}
test_data = {'traj': traj, 'energies': energies, 'forces': forces}

soap_params = {
        'n_max': 5,
        'l_max': 5,
        'r_cut': 5.0
        }

training_params = { 
        'lr': 0.01, 
        'model': 'simple'
        }

gp_model = GP_model_ase_traj(training_data, test_data, training_params, soap_params)

gp_model.train()
gp_model.test_performace()


In [50]:
type(gp_model)

fande.ase.gp_model.GP_model_ase_traj

### Dynamics simulation

In [ ]:
from sdynamics.dynamics import MDynamics

atoms = ... # initialize saddle structure

atoms.calc = gp_model.ase_calc() # get calc from the trained GP

md_runner = MDynamics(atoms) # instantiate class for running MD

md_runner.run() # run the MD

### Visualize:

In [48]:
from ase.visualize import view

# trj = trajectories[trj_files_basename[2]]
view(traj)

<Popen: returncode: None args: ['/home/dlb/anaconda3/envs/pyc/bin/python', '...>